# Time Series clustering

Ссылка на опрос -  https://otus.pw/sDfY/ 

## Зачем? 

- Найти похожие по динамике и паттернам группы временных рядов
- Финансовый анализ
- Поиск аномальных событий
- ...

## Проблемы

- Похожие паттерны могут находиться в разных участках временных рядов (начало/конец)
- Временные ряды разной длины
- Сложность вычислений

## Методы 

- Классический KMeans и прочие привычные алгоритмы кластеризации
    - на сырых данных, приведенных к одной длине (padding/sampling/...)
    - на признаках, извлеченных из временных рядов
    - вычислительно дешевый
- Dynamic Time Warping
    - позволяет работать с рядами разной длины
    - умеет находить паттерны в разных участках
    - вычислительно дорогой
- Time Series Embeddings
    - берем сеточку, засовываем временные ряды любой длины, на выходе получаем сжатые представления финксированной длины
    - кластеризуем эмбеддинги
    - profit

# Dynamic Time Warping

Хорошее объяснение происходящего - https://tslearn.readthedocs.io/en/stable/user_guide/dtw.html#dtw-softdtw


<img src="https://upload.wikimedia.org/wikipedia/commons/6/69/Euclidean_vs_DTW.jpg" width=400>

## Матрица попарных расстояний и Оптимальный путь

$$DTW(x, y) = min_{\pi} \sqrt{\sum_{(i,j)\in \pi} d(x_i, y_j)^2},$$

где $\pi = [\pi_0, ..., \pi_K]$ - это путь, удовлетворяющий набору условий:
- это лист парных индексов $\pi_k = (i_k, j_k)$, где  $0\leq i_k < n$ и $0\leq j_k < m$
- $\pi_0 = (0, 0)$ и $\pi_K = (n-1, m-1)$
- для всех $k>0$, $\pi_k=(i_k, j_k)$ и $\pi_{k-1}=(i_{k-1}, j_{k-1})$ удовлетворяют следующим неравенствам:
    - $i_{k-1} \leq i_k \leq i_{k-1} + 1$
    - $j_{j-1} \leq j_k \leq j_{k-1} + 1$
    

А если по-человечески:
- Путь - это индексы временных рядов
- Путь может ходить по диагонали, горизонтали и вертикали, главное, чтобы был неубывающим по индексам
- Стартуем всегда в начале временных рядов, заканчиваем всегда в конце

И для наглядности картинка:

<img src="https://tslearn.readthedocs.io/en/stable/_images/sphx_glr_plot_dtw_thumb.svg" width=400>

Итого наша цель - найти такой путь, который будет минимизировать некоторую метрику расстояния (чаще всего просто евклидову) между соответствующими участками временных рядов. 

Этот путь можно рассматривать как способ ~~натягивания совы на глобус~~ сопоставления участков временных рядов друг с другом безотносительно времени их появления. В результате, мы находим похожие по паттернам участки и сопоставляем их друг другу. 

Решается при помощи динамического программирования за $O(nm)$, где $n, m$ - длины рядов.

Еще можно навернуть ограничений на путь, чтобы особо долго не ходить, например, сказать, что особого пространства для маневров у нас не будет и можно смотреть только на несколько шагов вперед/назад:

<img src="https://tslearn.readthedocs.io/en/stable/_images/sakoe_chiba.png" width=200>